In [98]:
import G6_iris_recognition as iris
import os
import numpy as np
from time import time, sleep
import shutil
import cv2

In [116]:
def treat_img(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    h, w = img.shape
    cords = (18, 33)
    img_new = img
    
    ycord2 = img_new.shape[0]
    ycord = 0
    img_cord = cords

    while abs(ycord2 - ycord) > img_new.shape[0] * 0.70 and not img_new.shape[0] < img.shape[1] * 0.5:

      img_cord = (cords[0], img_cord[1] + 4)
      img_cord2 = cords
      x1, y1 = img_cord
      x2, y2 = img_cord2
      h, w = img_new.shape

      #Crop image
      crop_img = img_new[y1 : h - y2][x1 : w - x2]
      #Equalize image
      eq_img = cv2.equalizeHist(crop_img)
      #Blur image
      blur_img = cv2.medianBlur(eq_img, 25)
      #Detect bord
      bord_img = cv2.Canny(blur_img, 50, 100)
      minmax = np.argwhere(bord_img > 0)
      ycord,xcord = minmax.min(axis=0)
      ycord2,xcord2 = minmax.max(axis=0)

    #create a black background img 
    background = np.zeros((h, w), np.uint8)
    croppedImg = eq_img[ycord : ycord2 , xcord : xcord2]
    background[ ycord : ycord2, xcord : xcord2 ] = croppedImg

    croppedImg = background
    eq_img = cv2.cvtColor(eq_img, cv2.COLOR_GRAY2BGR)
    rec_img = cv2.rectangle(eq_img, (xcord,ycord), (xcord2,ycord2), (0,0,255), 3, cv2.LINE_AA)

    return croppedImg

In [112]:
shutil.rmtree("images_used", ignore_errors=True)
sleep(1)
os.mkdir("images_used")
folders = range(25)
img_dir = "images"

#Creat folder with the files we will use for the model
for i in folders:
    dir_name = str(i).zfill(4)
    if not os.path.isdir(f"images_used/{dir_name}"):
        os.mkdir(f"images_used/{dir_name}")
    for j in range(20):
        img_path = f"{dir_name}/{dir_name}_{str(j).zfill(3)}.bmp"
        img = cv2.imread(f"{img_dir}/{img_path}")
        cv2.imwrite(f"images_used/{img_path}", img)

In [119]:
shutil.rmtree("treated_images", ignore_errors=True)
sleep(1)
os.mkdir("treated_images")

path = "images_used"
dir_list = list()
#Create treated_images folder
for root, dirs, files in os.walk(path, topdown=False):
	for name in dirs:
		dir_list.append(os.path.join(root, name))
dir_len = len(dir_list)

if not os.path.isdir("treated_images"):
	os.mkdir("treated_images")

#Treat images and save files on treated_images folder
for i in range(dir_len):
	dir_name = str(i).zfill(4)
	if not os.path.isdir(f"treated_images/{dir_name}"):
		os.mkdir(f"treated_images/{dir_name}")
	for j in range(20):
		file_name = dir_name + "_" + str(j).zfill(3)
		img = cv2.imread(f"images_used/{dir_name}/{file_name}.bmp")
		img = treat_img(img)
		print(f"treated_images/{dir_name}/{file_name}.bmp")
		cv2.imwrite(f"treated_images/{dir_name}/{file_name}.bmp", img)


treated_images/0000/0000_000.bmp
treated_images/0000/0000_001.bmp
treated_images/0000/0000_002.bmp
treated_images/0000/0000_003.bmp
treated_images/0000/0000_004.bmp
treated_images/0000/0000_005.bmp
treated_images/0000/0000_006.bmp
treated_images/0000/0000_007.bmp
treated_images/0000/0000_008.bmp
treated_images/0000/0000_009.bmp
treated_images/0000/0000_010.bmp
treated_images/0000/0000_011.bmp
treated_images/0000/0000_012.bmp
treated_images/0000/0000_013.bmp
treated_images/0000/0000_014.bmp
treated_images/0000/0000_015.bmp
treated_images/0000/0000_016.bmp
treated_images/0000/0000_017.bmp
treated_images/0000/0000_018.bmp
treated_images/0000/0000_019.bmp
treated_images/0001/0001_000.bmp
treated_images/0001/0001_001.bmp
treated_images/0001/0001_002.bmp
treated_images/0001/0001_003.bmp
treated_images/0001/0001_004.bmp
treated_images/0001/0001_005.bmp
treated_images/0001/0001_006.bmp
treated_images/0001/0001_007.bmp
treated_images/0001/0001_008.bmp
treated_images/0001/0001_009.bmp
treated_im

In [120]:
#Treino
with open("model.pickle", "wb") as model:
    pass

iris.iris_model_train("treated_images", "model.pickle")

directory_list ['treated_images/0000', 'treated_images/0001', 'treated_images/0002', 'treated_images/0003', 'treated_images/0004']
name 0000
path_to_image treated_images/0000/0000_000.bmp
rectangle expression1
path_to_image treated_images/0000/0000_001.bmp
path_to_image treated_images/0000/0000_002.bmp
path_to_image treated_images/0000/0000_003.bmp
path_to_image treated_images/0000/0000_004.bmp
path_to_image treated_images/0000/0000_005.bmp
path_to_image treated_images/0000/0000_006.bmp
path_to_image treated_images/0000/0000_007.bmp
path_to_image treated_images/0000/0000_008.bmp
rectangle expression1
path_to_image treated_images/0000/0000_009.bmp
path_to_image treated_images/0000/0000_010.bmp
path_to_image treated_images/0000/0000_011.bmp
path_to_image treated_images/0000/0000_012.bmp
path_to_image treated_images/0000/0000_013.bmp
path_to_image treated_images/0000/0000_014.bmp
path_to_image treated_images/0000/0000_015.bmp
path_to_image treated_images/0000/0000_016.bmp
rectangle expres

['0004']

In [121]:
#Teste
correto = 0
errado = 0
treated_im_PATH = './treated_images'
for n_folder in os.listdir(treated_im_PATH):    
  for file in os.listdir(f'{treated_im_PATH}/{n_folder}'):
    result = iris.iris_model_test("model.pickle", f'{treated_im_PATH}/{n_folder}/{file}')
    if result == n_folder:
      correto += 1 
      print("Match", n_folder)
    else:
      print(result, n_folder)
      errado += 1

total = correto + errado
print("Acertos:", correto)
print("Erros: ", errado)
print("Porcentagem de acertos: ", correto/total)

rectangle expression1
unmatch 0000
unmatch 0000
unmatch 0000
unmatch 0000
unmatch 0000
unmatch 0000
unmatch 0000
unmatch 0000
rectangle expression1
unmatch 0000
unmatch 0000
unmatch 0000
unmatch 0000
unmatch 0000
unmatch 0000
unmatch 0000
unmatch 0000
rectangle expression1
unmatch 0000
unmatch 0000
unmatch 0000
unmatch 0000
unmatch 0001
unmatch 0001
unmatch 0001
unmatch 0001
rectangle expression1
unmatch 0001
unmatch 0001
unmatch 0001
unmatch 0001
unmatch 0001
rectangle expression1
unmatch 0001
unmatch 0001
rectangle expression1
unmatch 0001
unmatch 0001
unmatch 0001
unmatch 0001
unmatch 0001
unmatch 0001
unmatch 0001
unmatch 0001
unmatch 0001
unmatch 0002
0004 0002
unmatch 0002
unmatch 0002
unmatch 0002
unmatch 0002
unmatch 0002
unmatch 0002
unmatch 0002
unmatch 0002
0004 0002
unmatch 0002
unmatch 0002
rectangle expression1
unmatch 0002
rectangle expression1
unmatch 0002
rectangle expression1
unmatch 0002
rectangle expression1
unmatch 0002
unmatch 0002
0004 0002
unmatch 0002
rectangle